### Model Portfolio Max Sharpe Ratio

In [9]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from dotenv import load_dotenv
import os

# -----------------------------
# Setup & MT5 initialization
# -----------------------------
load_dotenv()
login = int(os.getenv("ACG_MT5_LOGIN"))
password = os.getenv("ACG_MT5_PASSWORD")
server = os.getenv("ACG_MT5_SERVER")

if not mt5.initialize(login=login, password=password, server=server):
    print("initialize() failed, error code =", mt5.last_error())
    quit()

# -----------------------------
# Config
# -----------------------------
risk_free_rate = 0.0  # annual risk-free rate

# FX pairs needed to convert index returns to USD
fx_map = {
    "GER30.pro": "EURUSD.pro",
    "JPN225.pro": "USDJPY.pro",   # flip sign
    "HK50.pro": "USDHKD.pro",     # flip sign
    "UK100.pro": "GBPUSD.pro",
    "EUSTX50.pro": "EURUSD.pro"
}

# -----------------------------
# Helpers
# -----------------------------
def fetch_mt5_data(symbol, days=60):
    utc_to = pd.Timestamp.now()
    utc_from = utc_to - pd.Timedelta(days=days)

    rates = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_D1, utc_from, utc_to)
    if rates is None:
        raise ValueError(f"Failed to fetch data for {symbol}")

    df = pd.DataFrame(rates)
    df["time"] = pd.to_datetime(df["time"], unit="s")
    df.set_index("time", inplace=True)
    return df

def calculate_log_returns(prices: pd.Series) -> pd.Series:
    return np.log(prices / prices.shift(1)).dropna()

def ewma_covariance(returns: pd.DataFrame, lambda_: float = 0.94) -> pd.DataFrame:
    r = returns.values
    T, N = r.shape

    init_window = min(5, T)
    cov = np.cov(r[:init_window].T)

    for t in range(init_window, T):
        x = r[t].reshape(-1, 1)
        cov = lambda_ * cov + (1.0 - lambda_) * (x @ x.T)

    return pd.DataFrame(cov, index=returns.columns, columns=returns.columns)

# -----------------------------
# Load factor signals
# -----------------------------
expected_returns_df = pd.read_csv("ai_factor_signal_acg.csv", sep="\t", header=None)
expected_returns_df.columns = ["asset", "factor_signal"]

# -----------------------------
# Z-score the factor signals
# -----------------------------
mean_signal = expected_returns_df["factor_signal"].mean()
std_signal = expected_returns_df["factor_signal"].std()

expected_returns_df["zscore_signal"] = (
    (expected_returns_df["factor_signal"] - mean_signal) / std_signal
)

expected_returns_dict = dict(
    zip(expected_returns_df["asset"], expected_returns_df["zscore_signal"])
)

symbols = list(expected_returns_dict.keys())

# -----------------------------
# Fetch price data for assets
# -----------------------------
data = {symbol: fetch_mt5_data(symbol) for symbol in symbols}

# -----------------------------
# Fetch FX data and compute FX log returns
# -----------------------------
fx_symbols = set(fx_map[s] for s in symbols if s in fx_map)
fx_data = {fx: fetch_mt5_data(fx) for fx in fx_symbols}

fx_returns = {}
for fx, df_fx in fx_data.items():
    lr = calculate_log_returns(df_fx["close"])

    if fx.startswith("USD"):
        lr = -lr

    fx_returns[fx] = lr

# -----------------------------
# Compute USD-adjusted log returns
# -----------------------------
asset_returns = {}

for symbol in symbols:
    idx_lr = calculate_log_returns(data[symbol]["close"])

    if symbol in fx_map:
        fx_symbol = fx_map[symbol]
        fx_lr = fx_returns[fx_symbol]

        combined = pd.concat([idx_lr, fx_lr], axis=1, join="inner")
        combined.columns = ["idx", "fx"]

        asset_returns[symbol] = combined["idx"] + combined["fx"]
    else:
        asset_returns[symbol] = idx_lr

# -----------------------------
# Build aligned returns matrix
# -----------------------------
returns = pd.DataFrame(asset_returns).dropna(how="any")

# -----------------------------
# Compute daily volatilities
# -----------------------------
daily_vols = returns.std().values  # vector aligned with symbols

# -----------------------------
# EWMA Covariance matrix
# -----------------------------
lambda_ = 0.94
cov_matrix = ewma_covariance(returns, lambda_=lambda_)
cov_matrix = cov_matrix + 1e-6 * np.eye(len(cov_matrix))

# -----------------------------
# Expected daily returns vector (volatility-scaled)
# -----------------------------
IC_estimate = 0.15  # assumed Information Coefficient
zscores = np.array([expected_returns_dict[symbol] for symbol in symbols])

if IC_estimate < 0:
    zscores = -zscores
    IC_estimate = abs(IC_estimate)

# Multiply by volatility
expected_returns = IC_estimate * zscores * daily_vols

# -----------------------------
# Max Sharpe optimization
# -----------------------------
def max_sharpe_ratio(expected_returns, cov_matrix, risk_free_rate=0.0):
    num_assets = len(expected_returns)

    def neg_sharpe(weights):
        port_ret = np.dot(weights, expected_returns)
        port_vol = np.sqrt(weights.T @ cov_matrix @ weights)
        if port_vol <= 0:
            return 1e6
        return -(port_ret - risk_free_rate) / port_vol

    constraints = [
        {"type": "ineq", "fun": lambda w: 1.0 - np.sum(w**2)}
    ]

    bounds = [(-1, 1)] * num_assets
    init_guess = np.ones(num_assets) / num_assets

    result = minimize(
        neg_sharpe,
        init_guess,
        bounds=bounds,
        constraints=constraints,
        method="SLSQP",
    )

    if not result.success:
        print("Warning: optimization did not fully converge:", result.message)

    weights = result.x
    daily_ret = np.dot(weights, expected_returns)
    daily_vol = np.sqrt(weights.T @ cov_matrix @ weights)

    return weights, daily_ret, daily_vol

# -----------------------------
# Run optimization
# -----------------------------
weights, daily_port_return, daily_port_vol = max_sharpe_ratio(
    expected_returns, cov_matrix, risk_free_rate=risk_free_rate
)

# -----------------------------
# Export optimal weights
# -----------------------------
weights_df = pd.DataFrame({
    "Symbol": symbols,
    "Weight": weights
})
weights_df.to_csv("acg_optimal_portfolio_weights.csv", header=False, index=False)
print("\nWeights exported to acg_optimal_portfolio_weights.csv")

# -----------------------------
# Annualized statistics
# -----------------------------
annual_port_return = daily_port_return * 252
annual_port_vol = daily_port_vol * np.sqrt(252)
annual_sharpe = (annual_port_return - risk_free_rate) / annual_port_vol

# -----------------------------
# Output
# -----------------------------

print("\n--- MODEL Portfolio Statistics ---")
print(f"Daily Return: {daily_port_return:.6%}")
print(f"Annualized Return: {annual_port_return:0.2%}")
print(f"Daily Volatility: {daily_port_vol:.6%}")
print(f"Annualized Volatility: {annual_port_vol:.2%}")
print(f"Annualized Sharpe Ratio: {annual_sharpe:.2f}")

# -----------------------------
# Shutdown MT5
# -----------------------------
mt5.shutdown()


Weights exported to acg_optimal_portfolio_weights.csv

--- MODEL Portfolio Statistics ---
Daily Return: 0.207070%
Annualized Return: 52.18%
Daily Volatility: 0.236048%
Annualized Volatility: 3.75%
Annualized Sharpe Ratio: 13.93


True

### Volatility Targeted Weights using EWMA estimates

In [10]:
import pandas as pd
import numpy as np

# ============================================================
# --- INPUTS: returns DataFrame must already exist ---
# ============================================================
# returns: T x N DataFrame of daily asset returns (log or simple, but consistent)
# Example:
# returns = pd.read_csv("your_returns_file.csv", index_col=0, parse_dates=True)

# ============================================================
# --- LOAD OPTIMAL WEIGHTS ---
# ============================================================
weights_df = pd.read_csv(
    "acg_optimal_portfolio_weights.csv",
    sep=",",
    header=None
)

weights_df.columns = ["asset", "weight"]

# Align weights to returns.columns order
weights_df = weights_df.set_index("asset").reindex(returns.columns)

# Extract weight vector
weights = weights_df["weight"].values

print("Loaded weights (aligned):")
print(weights)
print("\nAsset order:")
print(list(returns.columns))

# ============================================================
# --- EWMA COVARIANCE MATRIX (DAILY) ---
# ============================================================
lambda_ = 0.94  # RiskMetrics default
rets = returns.values
n = rets.shape[1]

# Start with sample covariance as initial value
ewma_cov = returns.cov().values.copy()

# EWMA update: use r_t at time t
for t in range(1, len(rets)):
    r = rets[t].reshape(-1, 1)
    ewma_cov = lambda_ * ewma_cov + (1 - lambda_) * (r @ r.T)

# Portfolio EWMA daily volatility
daily_portfolio_returns = returns @ weights
ewma_daily_vol = np.sqrt(weights.T @ ewma_cov @ weights)

# ============================================================
# --- UN-SCALED (BASE) PORTFOLIO STATISTICS ---
# ============================================================

ewma_annual_vol = ewma_daily_vol * np.sqrt(252)
ewma_annual_return = daily_portfolio_returns.mean() * 252

# Sharpe ratio (realized, ex-post)
ewma_sharpe = ewma_annual_return / ewma_annual_vol

print("\n--- EWMA VOLATILITY FORECAST ---")
print(f"Daily Volatility: {ewma_daily_vol:0.2%}")
print(f"Annual Volatility: {ewma_annual_vol:0.2%}")
print(f"Daily Portfolio Return (realized): {daily_portfolio_returns.mean():0.2%}")
print(f"Annual Portfolio Return (realized): {ewma_annual_return:0.2%}")
print(f"Sharpe Ratio (realized): {ewma_sharpe:0.4f}")

# ============================================================
# --- SCALE PORTFOLIO TO TARGET VOLATILITY USING EWMA ---
# ============================================================

target_annual_vol = 0.05  # e.g. 10% annual target

# Convert target annual vol to daily
target_daily_vol = target_annual_vol / np.sqrt(252)

# Scale factor in DAILY units
scale_factor = target_daily_vol / ewma_daily_vol
scaled_weights = weights * scale_factor

# Recompute volatility after scaling
scaled_daily_vol = np.sqrt(scaled_weights.T @ ewma_cov @ scaled_weights)
scaled_annual_vol = scaled_daily_vol * np.sqrt(252)

# Compute scaled returns
scaled_daily_returns = returns @ scaled_weights
scaled_avg_daily_return = scaled_daily_returns.mean()
scaled_annual_return = scaled_avg_daily_return * 252

# Sharpe ratio after scaling (realized)
scaled_sharpe = scaled_annual_return / scaled_annual_vol

print(f"\n--- SCALED PORTFOLIO (Target Vol = {target_annual_vol:.0%}) ---")
print(f"Factor: {scale_factor:.4f}")
print(f"Daily Volatility (EWMA): {scaled_daily_vol:0.2%}")
print(f"Annual Volatility (EWMA): {scaled_annual_vol:0.2%}")


print(f"\n--- EX-ANTE ---")
print(f"Daily Portfolio Return: {scaled_avg_daily_return:0.2%}")
print(f"Annual Portfolio Return: {scaled_annual_return:0.2%}")
print(f"Sharpe Ratio: {scaled_sharpe:0.4f}")

# --- EXPORT SCALED WEIGHTS TO CSV ---
scaled_df = pd.DataFrame({
    "asset": returns.columns,
    "scaled_weight": scaled_weights
})

scaled_df.to_csv("acg_scaled_weights.csv", index=False)
print("\nScaled weights exported to acg_scaled_weights.csv")

Loaded weights (aligned):
[ 0.12270584  0.01369907  0.09770516  0.10387317  0.00937471 -0.01894067
  0.12211265 -0.00722726 -0.17694737 -0.21710644 -0.07700722  0.05356171
 -0.05400828 -0.07140323 -0.63465827 -0.36289987 -0.49172628 -0.27564188]

Asset order:
['XAUUSD.pro', 'XAGUSD.pro', 'US30.pro', 'US500.pro', 'NAS100.pro', 'GER30.pro', 'EUSTX50.pro', 'UK100.pro', 'USDJPY.pro', 'USDCHF.pro', 'USOIL.pro', 'UKOIL.pro', 'JPN225.pro', 'HK50.pro', 'USDCAD.pro', 'GBPUSD.pro', 'AUDUSD.pro', 'NZDUSD.pro']

--- EWMA VOLATILITY FORECAST ---
Daily Volatility: 0.22%
Annual Volatility: 3.48%
Daily Portfolio Return (realized): 0.02%
Annual Portfolio Return (realized): 5.38%
Sharpe Ratio (realized): 1.5460

--- SCALED PORTFOLIO (Target Vol = 5%) ---
Factor: 1.4375
Daily Volatility (EWMA): 0.31%
Annual Volatility (EWMA): 5.00%

--- EX-ANTE ---
Daily Portfolio Return: 0.03%
Annual Portfolio Return: 7.73%
Sharpe Ratio: 1.5460

Scaled weights exported to acg_scaled_weights.csv


In [11]:
# Factor realization (1-day realized return weighted by portfolio)
factor_realization = (returns.tail(1).iloc[0] * scaled_weights)

# Align z-scores with factor realization
z = expected_returns_df.set_index("asset")["zscore_signal"]
z = z.loc[factor_realization.index]

# Compute correlation
corr = factor_realization.corr(z)
print(f"Factor Realization vs Signal Correlation: {corr:.4f}")


Factor Realization vs Signal Correlation: 0.2868


In [12]:

# 1. Get last 25 days of returns
last25 = returns.tail(25)

# 2. Factor signal (z-scores), aligned by asset
z = expected_returns_df.set_index("asset")["zscore_signal"]
z = z.loc[last25.columns]

# 3. Compute daily IC for each of the 25 days
ics = last25.apply(lambda r: r.corr(z), axis=1)

# 4. Ex-post IC over the last 25 days
expost_ic_25 = ics.mean()

print(f"Ex-post IC (25 days): {expost_ic_25:.4f}")



Ex-post IC (25 days): 0.1682


### Lot Sizing

In [13]:
import pandas as pd
import MetaTrader5 as mt5
import numpy as np

# ============================
# USER CONFIG
# ============================
EQUITY = 103543.79  # set your account equity

# FX-exempt symbols (no price in formula)
FX_EXEMPT = ["USDJPY.pro", "USDCHF.pro", "USDCAD.pro"]

# Global index → FX mapping
INDEX_FX_MAP = {
    "GER30.pro": "EURUSD.pro",
    "JPN225.pro": "USDJPY.pro",
    "HK50.pro": "USDHKD.pro",
    "UK100.pro": "GBPUSD.pro",
    "EUSTX50.pro": "EURUSD.pro"
}

# ============================
# MT5 INITIALIZATION
# ============================
mt5.initialize()

def get_latest_price(symbol):
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        return None
    return tick.ask

def fetch_prices(assets):
    return {a: get_latest_price(a) for a in assets}

def fetch_index_fx_rates():
    return {idx: get_latest_price(fx) for idx, fx in INDEX_FX_MAP.items()}

# ============================
# LOAD CSV FILES
# ============================
contract_df = pd.read_csv("acg_contract_size.csv")
weights_df = pd.read_csv("acg_scaled_weights.csv")

# Normalize column names
contract_df.columns = contract_df.columns.str.strip().str.lower()
weights_df.columns = weights_df.columns.str.strip().str.lower()

# Rename scaled_weight → weight
weights_df.rename(columns={"scaled_weight": "weight"}, inplace=True)

# Merge contract sizes + weights
df = contract_df.merge(weights_df, on="asset", how="left")

# ============================
# FETCH PRICES FROM MT5
# ============================
all_assets = df["asset"].tolist()
latest_prices = fetch_prices(all_assets)
index_fx_rates = fetch_index_fx_rates()

df["latest_price"] = df["asset"].map(latest_prices)

# ============================
# LOT SIZE CALCULATION
# ============================
def compute_lot(row):
    asset = row["asset"]
    weight = row["weight"]
    contract_size = row["contract_size"]
    price = row["latest_price"]

    # Missing weight or contract size
    if pd.isna(weight) or pd.isna(contract_size):
        return None

    # Contract size cannot be zero
    if contract_size == 0:
        return None

    # Rule 1: FX-exempt assets
    if asset in FX_EXEMPT:
        return (weight * EQUITY) / contract_size

    # Missing or zero price
    if price is None or price == 0:
        return None

    # Rule 2: Global index → convert to USD
    if asset in INDEX_FX_MAP:
        fx_rate = index_fx_rates.get(asset)
        if fx_rate is None or fx_rate == 0:
            return None
        # Invert FX rate for HK50 and JP225
        if asset in ["HK50.pro", "JPN225.pro"]:
            fx_rate = 1 / fx_rate

        price = price * fx_rate

        # After conversion, price still cannot be zero
        if price == 0:
            return None

    # Final safety check
    denominator = price * contract_size
    if denominator == 0:
        return None

    return (weight * EQUITY) / denominator

from decimal import Decimal, ROUND_HALF_UP

df["target_lot_size"] = df.apply(compute_lot, axis=1)

df["target_lot_size"] = df["target_lot_size"].apply(
    lambda x: Decimal(str(x)).quantize(Decimal("0.01"), rounding=ROUND_HALF_UP)
)

# ============================
# FETCH NET POSITIONS FROM MT5
# ============================

def fetch_net_positions():
    positions = mt5.positions_get()
    if positions is None:
        return {}

    net = {}

    for p in positions:
        symbol = p.symbol
        volume = p.volume if p.type == 0 else -p.volume  # BUY = +, SELL = -
        net[symbol] = net.get(symbol, 0) + volume

    return net

net_positions = fetch_net_positions()

df["current_holdings"] = pd.to_numeric(df["asset"].map(net_positions).fillna(0), errors="coerce")
df["target_lot_size"] = pd.to_numeric(df["target_lot_size"], errors="coerce")

df["difference"] = df["target_lot_size"] - df["current_holdings"]
df["difference"] = pd.to_numeric(df["difference"], errors="coerce").round(2)
# ============================
# OUTPUT
# ============================
print(df[["asset", "latest_price", "current_holdings", "target_lot_size", "difference"]].dropna())
df[["asset", "weight", "contract_size", "latest_price", "current_holdings", "target_lot_size"]]\
    .dropna()\
    .to_csv("acg_lot_sizes_output.csv", index=False)

# ============================
# GROSS TOTAL LOT SIZE
# ============================
df["abs_target_lot_size"] = df["target_lot_size"].abs()
df["current_holdings"] = df["current_holdings"].abs()
gross_target_lot_size = df["abs_target_lot_size"].sum()
gross_current_holdings = df["current_holdings"].sum()

print(f"\nGross Total Target Lot Size (absolute): {gross_target_lot_size:.2f}")
print(f"Gross Current Holdings (absolute): {gross_current_holdings:.2f}")

mt5.shutdown()

          asset  latest_price  current_holdings  target_lot_size  difference
0    AUDUSD.pro       0.67033               0.0            -1.09       -1.09
2    GBPUSD.pro       1.33959               0.0            -0.40       -0.40
3    NZDUSD.pro       0.57615               0.0            -0.71       -0.71
4    USDJPY.pro     158.11100               0.0            -0.26       -0.26
5    USDCHF.pro       0.80225               0.0            -0.32       -0.32
6    USDCAD.pro       1.38951               0.0            -0.94       -0.94
8    XAGUSD.pro      91.50600               0.0             0.00        0.00
9    XAUUSD.pro    4604.89000               0.0             0.04        0.04
10    USOIL.pro      59.62400               0.0            -1.92       -1.92
11    UKOIL.pro      63.82900               0.0             1.25        1.25
12    US500.pro    6970.31000               0.0             2.22        2.22
13     US30.pro   49529.35000               0.0             0.29        0.29

True

In [14]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform

# =========================
# CONFIG
# =========================
SYMBOLS = [
    "AUDUSD.pro","EURUSD.pro","GBPUSD.pro","NZDUSD.pro",
    "USDJPY.pro","USDCHF.pro","USDCAD.pro",
    "XAGUSD.pro","XAUUSD.pro",
    "USOIL.pro","UKOIL.pro",
    "US500.pro","US30.pro","NAS100.pro",
    "GER30.pro","JPN225.pro","HK50.pro",
    "UK100.pro","EUSTX50.pro"
]

TIMEFRAME = mt5.TIMEFRAME_W1   # weekly (recommended)
LOOKBACK = 104                 # 2 years
CORR_THRESHOLD = 0.8           # redundancy cutoff

# =========================
# INIT MT5
# =========================
if not mt5.initialize():
    raise RuntimeError("MT5 initialization failed")

# =========================
# DOWNLOAD DATA
# =========================
prices = {}

for symbol in SYMBOLS:
    rates = mt5.copy_rates_from_pos(symbol, TIMEFRAME, 0, LOOKBACK + 1)
    if rates is None:
        raise RuntimeError(f"Failed to get data for {symbol}")

    df = pd.DataFrame(rates)
    df["time"] = pd.to_datetime(df["time"], unit="s")
    df.set_index("time", inplace=True)

    prices[symbol] = df["close"]

mt5.shutdown()

# =========================
# BUILD RETURNS MATRIX
# =========================
price_df = pd.DataFrame(prices).dropna()
returns = price_df.pct_change().dropna()

# =========================
# CORRELATION → DISTANCE
# =========================
corr = returns.corr()

dist = np.sqrt(2 * (1 - corr))
dist_condensed = squareform(dist.values, checks=False)

# =========================
# CLUSTERING
# =========================
Z = linkage(dist_condensed, method="ward")

distance_threshold = np.sqrt(2 * (1 - CORR_THRESHOLD))
clusters = fcluster(Z, t=distance_threshold, criterion="distance")

cluster_map = pd.Series(clusters, index=returns.columns, name="cluster")

# =========================
# SELECT REPRESENTATIVES
# =========================
# Choose lowest volatility asset per cluster
vol = returns.std()

selected_assets = (
    vol
    .groupby(cluster_map)
    .idxmin()
    .tolist()
)

# =========================
# OUTPUT
# =========================
print("\nCLUSTER ASSIGNMENTS")
print(cluster_map.sort_values())

print("\nSELECTED NON-REDUNDANT ASSETS")
for s in selected_assets:
    print(s)




CLUSTER ASSIGNMENTS
USOIL.pro       1
UKOIL.pro       1
USDJPY.pro      2
USDCHF.pro      3
USDCAD.pro      4
NAS100.pro      5
US500.pro       5
US30.pro        6
JPN225.pro      7
GER30.pro       8
EUSTX50.pro     8
UK100.pro       9
HK50.pro       10
XAGUSD.pro     11
XAUUSD.pro     12
NZDUSD.pro     13
AUDUSD.pro     13
GBPUSD.pro     14
EURUSD.pro     14
Name: cluster, dtype: int32

SELECTED NON-REDUNDANT ASSETS
UKOIL.pro
USDJPY.pro
USDCHF.pro
USDCAD.pro
US500.pro
US30.pro
JPN225.pro
GER30.pro
UK100.pro
HK50.pro
XAGUSD.pro
XAUUSD.pro
AUDUSD.pro
GBPUSD.pro


In [18]:
import pandas as pd
import MetaTrader5 as mt5
from dotenv import load_dotenv
import os

# -----------------------------------
# Load credentials & initialize MT5
# -----------------------------------
load_dotenv()
login = int(os.getenv("ACG_MT5_LOGIN"))
password = os.getenv("ACG_MT5_PASSWORD")
server = os.getenv("ACG_MT5_SERVER")

if not mt5.initialize(login=login, password=password, server=server):
    print("MT5 initialization failed:", mt5.last_error())
    quit()

# -----------------------------------
# Load CSV
# -----------------------------------
df = pd.read_csv("acg_lot_sizes_output.csv")

# -----------------------------------
# Order sender (robust + unconstrained)
# -----------------------------------
def send_order(symbol, lot):

    # Direction
    order_type = mt5.ORDER_TYPE_BUY if lot > 0 else mt5.ORDER_TYPE_SELL
    lot = abs(lot)

    # Ensure symbol is enabled
    mt5.symbol_select(symbol, True)

    # Get fresh tick
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print(f"{symbol} → No tick data")
        return

    # Determine price
    price = tick.ask if order_type == mt5.ORDER_TYPE_BUY else tick.bid
    if price is None or price <= 0:
        print(f"{symbol} → Invalid price ({price})")
        return

    # Build order request
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": order_type,
        "price": price,
        "deviation": 1000,
        "magic": 1,
        "comment": "ICM auto-exec",
        "type_filling": mt5.ORDER_FILLING_IOC,  # most permissive
        "type_time": mt5.ORDER_TIME_GTC
    }

    # Send order
    result = mt5.order_send(request)

    # Print full result for debugging
    print(f"{symbol} {lot} → retcode: {result.retcode}")
    print("\n=== ORDER RESULT ===")
    print("Symbol:", symbol)
    print("Lot:", lot)
    print("Retcode:", result.retcode)
    print("Comment:", result.comment)
    print("Request ID:", result.request_id)
    print("Order:", result.order)
    print("Volume:", result.volume)
    print("Price:", result.price)
    print("Bid:", result.bid)
    print("Ask:", result.ask)
    print("Retcode External:", result.retcode_external)
print("====================\n")

# -----------------------------------
# Loop through CSV and execute
# -----------------------------------
for _, row in df.iterrows():
    symbol = row["asset"]
    lot = row["target_lot_size"]

    if lot != 0:
        send_order(symbol, lot)

# -----------------------------------
# Shutdown
# -----------------------------------
mt5.shutdown()


AUDUSD.pro 1.09 → retcode: 10027

=== ORDER RESULT ===
Symbol: AUDUSD.pro
Lot: 1.09
Retcode: 10027
Comment: AutoTrading disabled by client
Request ID: 0
Order: 0
Volume: 0.0
Price: 0.0
Bid: 0.0
Ask: 0.0
Retcode External: 0
GBPUSD.pro 0.4 → retcode: 10027

=== ORDER RESULT ===
Symbol: GBPUSD.pro
Lot: 0.4
Retcode: 10027
Comment: AutoTrading disabled by client
Request ID: 0
Order: 0
Volume: 0.0
Price: 0.0
Bid: 0.0
Ask: 0.0
Retcode External: 0
NZDUSD.pro 0.71 → retcode: 10027

=== ORDER RESULT ===
Symbol: NZDUSD.pro
Lot: 0.71
Retcode: 10027
Comment: AutoTrading disabled by client
Request ID: 0
Order: 0
Volume: 0.0
Price: 0.0
Bid: 0.0
Ask: 0.0
Retcode External: 0
USDJPY.pro 0.26 → retcode: 10027

=== ORDER RESULT ===
Symbol: USDJPY.pro
Lot: 0.26
Retcode: 10027
Comment: AutoTrading disabled by client
Request ID: 0
Order: 0
Volume: 0.0
Price: 0.0
Bid: 0.0
Ask: 0.0
Retcode External: 0
USDCHF.pro 0.32 → retcode: 10027

=== ORDER RESULT ===
Symbol: USDCHF.pro
Lot: 0.32
Retcode: 10027
Comment: A

True

In [16]:
lot

2.6